In [5]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import math

In [6]:
# --- 설정 ---
DATA_PATH = '/dataset/usr012/jmpark/Moudlar_data/data'
FILE_PATTERN = '*train.csv.gz'
DONE_CONDITIONS = [1, 2]  # 0이 아닌 값 (1 또는 2)
MA_WINDOW = 100           # 이동 평균 윈도우 (100개 에피소드 평균)

# 서브플롯 설정
PLOTS_PER_FIG = 6
PLOT_ROWS = 3
PLOT_COLS = 2

In [7]:
# 1. 파일 리스트 가져오기
file_list = sorted(glob.glob(os.path.join(DATA_PATH, FILE_PATTERN)))
num_files = len(file_list)

In [8]:
if num_files == 0:
    print(f"경고: {DATA_PATH} 경로에서 '{FILE_PATTERN}' 파일을 찾을 수 없습니다.")
else:
    print(f"총 {num_files}개의 파일을 찾았습니다. 그래프 생성을 시작합니다...")

총 38개의 파일을 찾았습니다. 그래프 생성을 시작합니다...


In [ ]:
# 2. 6개씩 묶어서 그래프 그리기
num_figures = math.ceil(num_files / PLOTS_PER_FIG)
file_iterator = iter(file_list) # 파일 리스트를 순회할 이터레이터

for fig_idx in range(num_figures):
    # 새 Figure 생성
    fig, axes = plt.subplots(PLOT_ROWS, PLOT_COLS, figsize=(15, 12))
    # 1D 배열로 변환하여 쉽게 접근
    axes = axes.flatten()
    
    # 3. 서브플롯 채우기 (최대 6개)
    for ax_idx in range(PLOTS_PER_FIG):
        try:
            # 다음 파일 가져오기
            file_path = next(file_iterator)
            filename = os.path.basename(file_path)
            ax = axes[ax_idx] # 현재 축 선택
            
            print(f"처리 중: {filename}...")

            # 4. 데이터 로드 (메모리 효율을 위해 필요한 컬럼만)
            try:
                df = pd.read_csv(
                    file_path,
                    compression='gzip',
                    usecols=['reward', 'done_code'] # ★★★ 핵심 ★★★
                )
            except Exception as e:
                print(f"  오류: {filename} 파일 로드 실패. {e}")
                ax.set_title(f"{filename}\n(파일 로드 실패)", color='red')
                continue

            episode_rewards = []
            episode_end_steps = []
            current_episode_reward = 0.0

            # 5. 데이터를 순회하며 에피소드 리워드 계산
            for step, (reward, done_code) in enumerate(df.itertuples(index=False)):
                current_episode_reward += reward
                
                # done_code가 0이 아니면 (1 또는 2) 에피소드 종료
                if done_code in DONE_CONDITIONS:
                    episode_rewards.append(current_episode_reward)
                    episode_end_steps.append(step) # 종료 시점의 '전체 스텝'
                    current_episode_reward = 0.0 # 리워드 초기화

            if not episode_rewards:
                print(f"  경고: {filename}에 완료된 에피소드가 없습니다.")
                ax.set_title(f"{filename}\n(완료된 에피소드 없음)", color='orange')
                continue

            # 6. 이동 평균 계산 및 그래프 그리기
            rewards_series = pd.Series(episode_rewards)
            moving_avg = rewards_series.rolling(window=MA_WINDOW, min_periods=1).mean()

            ax.plot(episode_end_steps, moving_avg) # X축: 에피소드 종료 스텝, Y축: 이동 평균 리워드
            ax.set_title(filename)
            ax.set_xlabel("Timestep")
            ax.set_ylabel(f"Episode Reward (MA {MA_WINDOW})")
            ax.grid(True)

        except StopIteration:
            # 처리할 파일이 더 없으면 남은 subplot을 끈다
            axes[ax_idx].axis('off')

    # Figure 저장 및 출력
    fig_filename = f"training_rewards_figure_{fig_idx + 1}.png"
    plt.suptitle(f"Training Progress (Figure {fig_idx + 1}/{num_figures})", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # suptitle과 겹치지 않게
    plt.savefig(fig_filename)
    print(f"--- {fig_filename} 저장 완료 ---")
    plt.show() # 주피터 노트북 등이 아니면 주석 처리
    plt.close(fig) # 메모리 해제

print("모든 그래프 생성이 완료되었습니다.")